In [0]:
access_key = "cEHFD5KQSB2yfcAAGPnyNt+hBFoniH5UCgquqZKmh1yypAwCe26MlKtTpLHX7dW2apc3Uh12571B+AStcnMAfg=="
endpoints = "https://adlsteam3.dfs.core.windows.net/"

In [0]:
spark.conf.set("fs.azure.account.key.adlsteam3.dfs.core.windows.net", access_key)

In [0]:
claims_df = spark.read.format("delta").load(
    "abfss://silver@adlsteam3.dfs.core.windows.net/claims"
)
claims_df.createOrReplaceTempView("claims_silver")


shipments_df = spark.read.format("delta").load(
    "abfss://silver@adlsteam3.dfs.core.windows.net/shipments"
)
shipments_df.createOrReplaceTempView("shipments_silver")



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7539119822929449>, line 9
      4 claims_df.createOrReplaceTempView("claims_silver")
      6 deliverylogs_df = spark.read.format("delta").load(
      7     "abfss://silver@adlsteam3.dfs.core.windows.net/delivery_logs"
      8 )
----> 9 deliverylogs_df.createOrReplaceTempView("delivery_logs_silver")
     11 shipments_df = spark.read.format("delta").load(
     12     "abfss://silver@adlsteam3.dfs.core.windows.net/shipments"
     13 )
     14 shipments_df.createOrReplaceTempView("shipments_silver")

File /databricks/spark/python/pyspark/sql/connect/dataframe.py:2189, in DataFrame.createOrReplaceTempView(self, name)
   2185 def createOrReplaceTempView(self, name: str) -> None:
   2186     command = plan.CreateView(
   2187         child=self._plan, name=name, is_global=False, replace=True
   2188     ).command(session=self._se

In [0]:

deliverylogs_df = spark.read.format("delta").load(
    "abfss://silver@adlsteam3.dfs.core.windows.net/delivery_logs"
)
deliverylogs_df.createOrReplaceTempView("delivery_logs_silver")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7539119822929453>, line 4
      1 deliverylogs_df = spark.read.format("delta").load(
      2     "abfss://silver@adlsteam3.dfs.core.windows.net/delivery_logs"
      3 )
----> 4 deliverylogs_df.createOrReplaceTempView("delivery_logs_silver")

File /databricks/spark/python/pyspark/sql/connect/dataframe.py:2189, in DataFrame.createOrReplaceTempView(self, name)
   2185 def createOrReplaceTempView(self, name: str) -> None:
   2186     command = plan.CreateView(
   2187         child=self._plan, name=name, is_global=False, replace=True
   2188     ).command(session=self._session.client)
-> 2189     _, _, ei = self._session.client.execute_command(command, self._plan.observations)
   2190     self._execution_info = ei

File /databricks/spark/python/pyspark/sql/connect/client/core.py:1481, in SparkConnectClient.execute_command(self

In [0]:
%sql
SELECT claim_status, COUNT(*) AS Total_Claims

FROM claims_silver

GROUP BY claim_status

ORDER BY Total_Claims DESC;
 

claim_status,Total_Claims
Rejected,127
Pending,96
Approved,77


In [0]:
%sql
SELECT 
    claim_status,
    COUNT(*) AS Total_Claims,
    AVG(amount_claimed) AS Avg_Claim_Amount,
    SUM(amount_claimed) AS Total_Claim_Amount
FROM claims_silver
GROUP BY claim_status
ORDER BY Total_Claims DESC;

claim_status,Total_Claims,Avg_Claim_Amount,Total_Claim_Amount
Rejected,127,506.09968503937006,64274.659999999996
Pending,96,590.7633333333333,56713.28
Approved,77,587.1862337662338,45213.340000000004


In [0]:
%sql
SELECT 
    s.shipment_id,
    s.delivery_date,
    s.ship_date,
    DATEDIFF(s.ship_date, s.delivery_date) AS delivery_days,
    CASE 
        WHEN DATEDIFF(s.ship_date, s.delivery_date) > (
            SELECT percentile_approx(delivery_duration_days, 0.5)
            FROM deliverylogs_silver
        )
        THEN 'Delayed'
        ELSE 'On Time'
    END AS status
FROM shipments_silver s;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7539119822929452>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "SELECT \n    s.shipment_id,\n    s.delivery_date,\n    s.ship_date,\n    DATEDIFF(s.ship_date, s.delivery_date) AS delivery_days,\n    CASE \n        WHEN DATEDIFF(s.ship_date, s.delivery_date) > (\n            SELECT percentile_approx(delivery_duration_days, 0.5)\n            FROM deliverylogs_silver\n        )\n        THEN 'Delayed'\n        ELSE 'On Time'\n    END AS status\nFROM shipments_silver s;\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # wh